<a href="https://colab.research.google.com/github/vhuni/hackathon_chatbot/blob/main/akero_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
# !pip install rasa

In [57]:
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 43.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [58]:
!pip install nest_asyncio

In [59]:
import os
import rasa
import nest_asyncio

nest_asyncio.apply()

print("event loop is ready")

event loop is ready


In [60]:
from rasa.cli.scaffold import create_initial_project

project = 'akero_chatbot'
create_initial_project(project)

In [61]:
os.chdir(project)
print(os.listdir("."))

['endpoints.yml', 'tests', 'actions', 'credentials.yml', 'domain.yml', 'data', 'config.yml']


In [62]:
config = "config.yml"
training_files = "data/"
domain = "domain.yml"
output = "models/"


In [63]:
%%writefile config.yml

recipe: default.v1
assistant_id: example_bot
language: en

pipeline:
# will be selected by the Suggested Config feature

policies:
- name: MemoizationPolicy
- name: TEDPolicy
  max_history: 5
  epochs: 10



Overwriting config.yml


In [72]:
%%writefile domain.yml


intents:
  - greet
  - akero_inquiry
  - campaign_inquiry
  - tactic_inquiry
  - affirm

slots:
  Akero:
    type: text
    mappings:
    - type: from_text
      intent: akero_inquiry

  # Akero:
  #   type: unfeaturized
    # values:
    #     - akero
    #     - marketing
    #     - company
    #     - website
    #     - net natives
    #     - product
    #     - application

  Campaign:
    type: text
    mappings:
    - type: from_text
      intent: campaign_inquiry
  # Campaign:
  #   type: unfeaturized
    # values:
    #     - campaign
    #     - client
    #     - campaign strategist
    #     - advertising
    #     - brand promotion
    #     - initiative
    #     - promotion
  Tactic:
    type: text
    mappings:
    - type: from_text
      intent: tactic_inquiry
  # Tactic:
  #   type: unfeaturized
    # values:
    #     - media marketing
    #     - content
    #     - ads
    #     - paid media
    #     - assets


entities:
  - Akero
  - Campaign
  - Tactic


actions:
  - utter_greet
  - utter_akero_req
  - utter_tactic_req
  - utter_campaign_req
  - utter_affirm
  - __main__.ApiAction

responses:
  utter_greet:
  - text: "Hi how you doing"
  utter_akero_req:
  - text: "The student advertising and conversion platform powered by AI supported by Higher Education experts."
  # utter_akero_req:
  # - text: "Akero was created.."

  utter_tactic_req:
  - text: "Hello world this is a tactic"

  utter_campaign_req:
  - text: "Hello world this is a campaign"

  utter_affirm:
  - text: "Welcome"

session_config:
  session_expiration_time: 60  # value in minutes
  carry_over_slots_to_new_session: true

Overwriting domain.yml


In [65]:
%%writefile data/nlu.yml

version: "3.1"

nlu:

- intent: greet
  examples: |
    - hey
    - hello there
    - hi
    - hello
    - good morning
    - good evening
    - hey there
    - let's go
    - hey dude
    - goodmorning
    - goodevening
    - good afternoon

- intent: affirm
  examples: |
    - great
    - OK
    - okay
    - thank you
    - thanks
    - yes, thanks
    - cheers
    - thanks a lot
    - thank you very much

- intent: akero_inquiry
  examples: |
    - What is akero ?
    - Is akero a marketing application
    - Is akero created by NetNatives

- intent: campaign_inquiry
  examples: |
    - What are campaigns?
    - Could you explain what campaigns are?

- intent: campaign_tactic
  examples: |
    - Who creates tactics?
    - what are tactics?

Overwriting data/nlu.yml


In [66]:
# print(config, training_files, domain, output)

In [67]:
%%writefile data/rules.yml

version: "3.1"

rules:

- rule: Say goodbye anytime the user says goodbye
  steps:
  - intent: affirm
  - action: utter_affirm

Overwriting data/rules.yml


In [68]:
%%writefile data/stories.yml

## path 1
stories:
- story : akero_inq_1
  steps:
  - intent: greet
  - action : utter_greet
  - intent : akero_inquiry
  - action : utter_akero_req
  - intent : affirm
  - action : utter_affirm

- story : campaign_inq_1
  steps:
  - intent: greet
  - action : utter_greet
  - intent : campaign_inquiry
  - action : utter_campaign_req
  - intent : affirm
  - action : utter_affirm

Overwriting data/stories.yml


In [73]:
## Training the data

model_path = rasa.train(domain, config, [training_files], output)
print(model_path)

/usr/local/lib/python3.10/dist-packages/rasa/shared/core/slot_mappings.py:224: UserWarning: Slot auto-fill has been removed in 3.0 and replaced with a new explicit mechanism to set slots. Please refer to https://rasa.com/docs/rasa/domain#slots to learn more.
  rasa.shared.utils.io.raise_warning(


The configuration for pipeline was chosen automatically. It was written into the config file at 'config.yml'.


/usr/local/lib/python3.10/dist-packages/rasa/shared/utils/io.py:99: UserWarning: 'RulePolicy' is not included in the model's policy configuration. Default intents such as 'restart' and 'back' will not trigger actions 'action_restart' and 'action_back'.
  More info at https://rasa.com/docs/rasa/default-actions
/usr/local/lib/python3.10/dist-packages/rasa/shared/utils/io.py:99: UserWarning: Found rule-based training data but no policy supporting rule-based data. Please add `RulePolicy` or another rule-supporting policy to the `policies` section in `config.yml`.
  More info at https://rasa.com/docs/rasa/rules
Processed trackers: 100%|██████████| 2/2 [00:00<00:00, 110.24it/s, # action=11]
Processed actions: 11it [00:00, 602.83it/s, # examples=11]
/usr/local/lib/python3.10/dist-packages/rasa/utils/train_utils.py:530: UserWarning: constrain_similarities is set to `False`. It is recommended to set it to `True` when using cross-entropy loss.
  rasa.shared.utils.io.raise_warning(
/usr/local/lib

Your Rasa model is trained and saved at 'models/20240418-153215-merciless-swab.tar.gz'.
TrainingResult(model='models/20240418-153215-merciless-swab.tar.gz', code=0, dry_run_results=None)


<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead


In [70]:
!rasa run actions & rasa shell

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


/usr/local/lib/python3.10/dist-packages/rasa/core/tracker_store.py:1044: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
/usr/local/lib/python3.10/dist-packages/rasa/core/tracker_store.py:1044: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades pr

In [75]:
from rasa.jupyter import chat

endpoints = None
chat(model_path.model, endpoints)

Your bot is ready to talk! Type your messages here or send '/stop'.
thank you
2024-04-18 15:37:45 [debug    ] processor.actions.policy_prediction prediction_events=[]
2024-04-18 15:37:45 [debug    ] processor.actions.log          action_name=action_session_start rasa_events=[<rasa.shared.core.events.SessionStarted object at 0x7e40ba3a5150>, ActionExecuted(action: action_listen, policy: None, confidence: None)]
2024-04-18 15:37:45 [debug    ] processor.slots.log            slot_values=	Akero: None
	Campaign: None
	Tactic: None
	session_started_metadata: None
2024-04-18 15:37:46 [debug    ] processor.message.parse        parse_data_entities=[] parse_data_intent={'name': 'affirm', 'confidence': 1.0} parse_data_text=thank you
2024-04-18 15:37:46 [debug    ] processor.extract.slots        action_extract_slot=action_extract_slots len_extraction_events=0 rasa_events=[]
2024-04-18 15:37:46 [debug    ] memoization.predict.actions    tracker_states=[{}, {'user': {'intent': 'affirm'}, 'prev_actio

KeyboardInterrupt: 